# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [26]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [27]:
# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [28]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [29]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [30]:
# Make a connection to a Cassandra instance on local machine 

from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [31]:
# create user_data keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS user_data 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [32]:
# set session keyspace to user_data
try:
    session.set_keyspace("user_data")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [33]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## create session_library table, composite primary key to guarantee unique entries and allow for specified query

query = """
        CREATE TABLE IF NOT EXISTS song_info_by_session
        (session_id int, item_in_session int,
        artist_name text, song_length text,  song_name text, 
        PRIMARY KEY ((session_id, item_in_session)))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)      

In [34]:
file = 'event_datafile_new.csv'

## insert relevant columns from csv file into Cassandra DB. line[0]: artist_name, line[3]: item_in_session, 
## line[5]: song_length, line[8]: session_id, line[9]: song_name
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_info_by_session (session_id, item_in_session, artist_name, song_length, song_name)"
        query = query + " values (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[5], line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [40]:
## execute SELECT statement to verify the data was entered into the table

query = "select artist_name, song_name, song_length from song_info_by_session \
        where session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# use PrettyTable for more readable output
table = PrettyTable(['artist_name', 'song_name', 'song_length'])
for row in rows:
    table.add_row([row.artist_name, row.song_name, row.song_length])
print(table)

+-------------+---------------------------------+-------------+
| artist_name |            song_name            | song_length |
+-------------+---------------------------------+-------------+
|  Faithless  | Music Matters (Mark Knight Dub) |   495.3073  |
+-------------+---------------------------------+-------------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [38]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## create user_session_library table. Composite user_id/session_id primary key with item_in_session clustering key. Will order
## user_id and session_id entries by item_in_session, potentially dropping songs listened to multiple times in the same session, but
## assuming this is nonessential data for the query.  
query = """
        CREATE TABLE IF NOT EXISTS user_info_by_id_and_session
        (user_id int, session_id int, item_in_session int,
        artist_name text, song_name text,
        first_name text, last_name text,  
        PRIMARY KEY ((user_id, session_id), item_in_session) )
        with clustering order by (item_in_session DESC);
        """
try:
    session.execute(query)
except Exception as e:
    print(e)           

In [39]:
file = 'event_datafile_new.csv'

## insert relevant columns from csv file into Cassandra DB. line[10]: user_id, line[8]: session_id, line[3]: item_in_session
## line[0]: artist_name, line[9]: song_name, line[1]: first_name, line[4]: last_name 
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
                insert into user_info_by_id_and_session
                (user_id, session_id, item_in_session, artist_name, song_name, first_name, last_name)
                """
        query = query + " values (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [41]:
## execute SELECT statement to verify the data was entered into the table

query = "select artist_name, song_name, first_name, last_name \
        from user_info_by_id_and_session \
        where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# use PrettyTable for more readable output
table = PrettyTable(['artist_name', 'song_name', 'first_name', 'last_name'])
for row in rows:
    table.add_row([row.artist_name, row.song_name, row.first_name, row.last_name])
print(table)

+-------------------+------------------------------------------------------+------------+-----------+
|    artist_name    |                      song_name                       | first_name | last_name |
+-------------------+------------------------------------------------------+------------+-----------+
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


In [42]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## only requires one entry for each user who listened to each song, primary key is therefore a composite of song/user_id
## so that only most recent session will be saved in memory, avoiding unnecessary duplicate entries and optimizing memory use
query = """
        CREATE TABLE IF NOT EXISTS user_info_by_song_and_id
        (song_name text, user_id int,
        first_name text, last_name text, 
        PRIMARY KEY(song_name, user_id))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)                

In [43]:
file = 'event_datafile_new.csv'

## insert relevant columns from csv file into Cassandra DB. line[9]: song_name, line[10]: user_id,
## line[1]: first_name, line[4]: last_name
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
                insert into user_info_by_song_and_id
                (song_name, user_id, first_name, last_name)
                """
        query = query + " values (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [45]:
## execute SELECT statement to verify the data was entered into the table

query = "select first_name, last_name \
        from user_info_by_song_and_id \
        where song_name='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# use PrettyTable for more readable output
table = PrettyTable(['first_name', 'last_name'])
for row in rows:
    table.add_row([row.first_name, row.last_name])
print(table)

+------------+-----------+
| first_name | last_name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [46]:
## TO-DO: Drop the table before closing out the sessions
query = "DROP TABLE song_info_by_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

query2 = "DROP TABLE user_info_by_id_and_session"
try:
    session.execute(query2)
except Exception as e:
    print(e)
    
query3 = "DROP TABLE user_info_by_song_and_id"
try:
    session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [47]:
session.shutdown()
cluster.shutdown()